# SAM2 Background Remover - Click to Select

**Setup:** Runtime → Change runtime type → GPU

In [ ]:
# Install everything
!pip install -q numpy==1.24.3 torch torchvision opencv-python pillow gradio==3.50.2
!git clone -q https://github.com/facebookresearch/segment-anything-2.git
!cd segment-anything-2 && pip install -q -e .
!mkdir -p sam2_checkpoints && cd sam2_checkpoints && wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt

import sys
sys.path.append('/content/segment-anything-2')
print("✅ Ready!")

In [ ]:
# Setup
import os
import torch
import numpy as np
from PIL import Image, ImageDraw
import gradio as gr
from google.colab import drive
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

# Mount Drive
drive.mount('/content/drive')
OUTPUT_DIR = '/content/drive/MyDrive/furniture_images/background_removed'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load SAM2
device = "cuda" if torch.cuda.is_available() else "cpu"
sam2 = build_sam2("sam2_hiera_l.yaml", "sam2_checkpoints/sam2_hiera_large.pt", device=device)
predictor = SAM2ImagePredictor(sam2)
print(f"✅ Using {device}")

In [ ]:
# Click Interface
points_list = []
current_image = None

def handle_click(image, evt: gr.SelectData, point_type):
    global current_image, points_list
    
    if image is None:
        return None, "Please upload an image first"
    
    # Store original image
    if current_image is None:
        current_image = image.copy()
    
    # Get click coordinates
    x, y = evt.index[0], evt.index[1]
    
    # Add point
    label = 1 if point_type == "Include (Green)" else 0
    points_list.append({"x": x, "y": y, "label": label})
    
    # Draw all points on image
    img = current_image.copy()
    draw = ImageDraw.Draw(img)
    
    for p in points_list:
        color = 'green' if p['label'] == 1 else 'red'
        # Draw circle
        draw.ellipse([p['x']-8, p['y']-8, p['x']+8, p['y']+8], 
                    fill=color, outline='white', width=2)
    
    return img, f"Added {'include' if label == 1 else 'exclude'} point at ({x}, {y}). Total points: {len(points_list)}"

def process_image():
    global current_image, points_list
    
    if current_image is None:
        return None, "Please upload an image"
    
    if not points_list:
        return None, "Please click on the image to add points"
    
    # Setup predictor
    predictor.set_image(np.array(current_image))
    
    # Prepare points
    coords = np.array([[p['x'], p['y']] for p in points_list])
    labels = np.array([p['label'] for p in points_list])
    
    # Generate mask
    masks, scores, _ = predictor.predict(
        point_coords=coords,
        point_labels=labels,
        multimask_output=True
    )
    
    # Use best mask
    mask = masks[np.argmax(scores)]
    
    # Create transparent result
    result = Image.new("RGBA", current_image.size, (0, 0, 0, 0))
    result.paste(current_image, (0, 0))
    alpha = Image.fromarray((mask * 255).astype(np.uint8))
    result.putalpha(alpha)
    
    # Save
    filename = f"furniture_{len(os.listdir(OUTPUT_DIR))}_nobg.png"
    save_path = os.path.join(OUTPUT_DIR, filename)
    result.save(save_path)
    
    return result, f"✅ Saved to {save_path}"

def clear_points():
    global points_list, current_image
    points_list = []
    if current_image is not None:
        return current_image, "Points cleared"
    return None, "Points cleared"

def reset_all():
    global points_list, current_image
    points_list = []
    current_image = None
    return None, None, "Reset complete"

# Create Gradio Interface
with gr.Blocks(title="SAM2 Click Interface") as app:
    gr.Markdown("""
    # 🖱️ Click to Remove Background
    
    1. Upload image
    2. **Click directly on the image** to add points
    3. Green = keep this area, Red = remove this area
    4. Click 'Remove Background' when done
    """)
    
    with gr.Row():
        with gr.Column():
            # Clickable image
            input_image = gr.Image(
                label="Click on the image to add points",
                type="pil",
                interactive=True
            )
            
            # Point type selector
            point_type = gr.Radio(
                choices=["Include (Green)", "Exclude (Red)"],
                value="Include (Green)",
                label="Point Type"
            )
            
            # Buttons
            with gr.Row():
                clear_btn = gr.Button("Clear Points")
                reset_btn = gr.Button("Reset All")
            
            process_btn = gr.Button("🎨 Remove Background", variant="primary", size="lg")
            
            status = gr.Textbox(label="Status", interactive=False)
            
        with gr.Column():
            output_image = gr.Image(
                label="Result (transparent background)",
                type="pil"
            )
            
            gr.Markdown("""
            ### Tips:
            - Click on the furniture to add green points (keep)
            - Click on background to add red points (remove)
            - Usually 2-3 green points on furniture is enough
            - Add red points if background is included
            """)
    
    # Connect events
    input_image.select(
        fn=handle_click,
        inputs=[input_image, point_type],
        outputs=[input_image, status]
    )
    
    clear_btn.click(
        fn=clear_points,
        outputs=[input_image, status]
    )
    
    reset_btn.click(
        fn=reset_all,
        outputs=[input_image, output_image, status]
    )
    
    process_btn.click(
        fn=process_image,
        outputs=[output_image, status]
    )

# Launch with public URL
app.launch(share=True)